# Keras Input 

## Input is NOT a Layer

Note that ```keras.Input``` is NOT in ```keras.layer``` package as it is not a Layer but a **Tensor**,

* [Input Object](https://keras.io/api/layers/core_layers/input/)

> Input() is used to instantiate a Keras **tensor**.



## Input Shape

Because the preprocessing normalization layer is the first layer to take inputs, it requires ```input_shape```.

* [Keras input explanation: input_shape, units, batch_size, dim, etc](https://stackoverflow.com/a/44748370/4281353)

> Example: if you have 30 images of 50x50 pixels in RGB (3 channels), the shape of your input data is (30,50,50,3). 
> 
> Keras demands the input shape at the first layer only, and Keras ignores the first dimension of the input data, which is the batch size. Your model should be able to deal with any batch size, so you define only the other dimensions:
> ```
> #regardless of how many images I have, each image has this shape   
> input_shape = (50,50,3)
> ```


# Regularizer

* [What is the difference between kernel, bias, and activity regulizers](https://stats.stackexchange.com/a/383326/105137)

> With a neural network regression equation ```𝑦=𝑊𝑥+𝑏```
> * Kernel Regularizer: Tries to reduce the weights 𝑊 (excluding bias).
> * Bias Regularizer: Tries to reduce the bias 𝑏
> * Activity Regularizer: Tries to reduce the layer's output ```𝑦``` thus will reduce the weights and adjust bias so ```𝑊𝑥+𝑏``` is smallest.
> 
> Usually, if you have no prior on the distribution that you wish to model, you would only use the kernel regularizer, since a large enough network can still model your function even if the regularization on the weights are big.
> 
> If you want the output function to pass through (or have an intercept closer to) the origin, you can use the bias regularizer.